In [1]:
!pip install tqdm
!pip install statsmodels


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
import sys
import os
sys.path.insert(1,os.path.join(os.getcwd(),'CandC'))
import certainty_stats
import certainty

2024-08-12 14:43:47.838085: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-12 14:43:48.527158: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
X=torch.randint(0,100,(1000,100))

In [4]:
prob_row = (lambda x: x/x.sum())

In [5]:
pX = torch.vmap(prob_row)(X)

In [6]:
y=torch.randint(0,100,(1000,))

In [7]:
xPred=X.max(1)[1]

In [8]:
halfsies= torch.cat([xPred[0:500],y[500:]])

In [9]:
halfsies.shape

torch.Size([1000])

In [10]:
certaintyscoreX = certainty.get_certainty(pX)[1] 

In [11]:
certaintyscoreX.shape

torch.Size([1000, 1])

In [12]:
cc1=certainty_stats.component_competence(pX,xPred)
cc2=certainty_stats.component_competence(pX,halfsies)
cc3=certainty_stats.component_competence(pX,y)

In [13]:
print('component competence on all predictions is {}\n component competence on half predictions is {}\n true component competence is {}'.format(cc1,cc2,cc3))

component competence on all predictions is 0.009951982647180557
 component competence on half predictions is 0.005319703370332718
 true component competence is 0.00033201370388269424


Let's rig an example for comparison

In [14]:
A= torch.Tensor([[1,0,0],[1,0,0],[.5,.4,.1],[.6,.2,.2],[.4,.3,.3],[1/3,1/3,1/3]])

In [15]:
APred = A.max(1)[1]

In [16]:
APred

tensor([0, 0, 0, 0, 0, 0])

In [17]:
AObs = torch.Tensor([0,0,0,0,0,1]).int()

In [18]:
certaintscoreA= certainty.get_certainty(A)[1]

In [19]:
certaintscoreA

tensor([[1.0000],
        [1.0000],
        [0.1000],
        [0.4000],
        [0.1000],
        [0.0000]])

In [20]:
certainty_stats.component_competence(A,APred)

tensor(0.3056)

In [21]:
certainty_stats.component_competence(A,AObs)

tensor(0.3056)

In [22]:
import pandas as pd

In [23]:
df1 = pd.DataFrame({'certainty_score':certaintyscoreX.flatten().numpy(), 'prediction':xPred, 'classification':xPred})
df2 = pd.DataFrame({'certainty_score':certaintyscoreX.flatten().numpy(), 'prediction':xPred, 'classification':halfsies})
df3 = pd.DataFrame({'certainty_score':certaintyscoreX.flatten().numpy(), 'prediction':xPred, 'classification':y})
df4 = pd.DataFrame({'certainty_score':certaintscoreA.flatten().numpy(), 'prediction':APred, 'classification':AObs})

In [24]:
predictive_ = (lambda x: 'TP' if x.prediction==x.classification else 'FP')

In [25]:
df1['predictive_status']=df1.apply(predictive_,axis=1)
df2['predictive_status']=df2.apply(predictive_,axis=1)
df3['predictive_status']=df3.apply(predictive_,axis=1)
df4['predictive_status']=df4.apply(predictive_,axis=1)

In [26]:
ec1= certainty_stats.empirical_competence(df1)
ec2= certainty_stats.empirical_competence(df2)
ec3= certainty_stats.empirical_competence(df3)
ec4= certainty_stats.empirical_competence(df4)

In [27]:
ec1

0.0001841806173324585

In [28]:
ec2

9.744085371494293e-06

In [29]:
ec3

-0.00017925770580768586

In [30]:
ec4

0.4333333174387614